# Gazeta do Povo - Governadores

Link Governadores da Bahia
+ https://especiais.gazetadopovo.com.br/eleicoes/2018/resultados/candidatos-mais-votados-para-governador-da-ba/


# Import and Constants

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os 

# Snippets

In [3]:
def convert_long_comment_in_string(long_txt):
    """
    Converter uma lsita grande dos attrs que queremos filtrar, pois selecionei manualmente
    """
    return list(map(lambda x: x.strip(), long_txt.split('\n')[1:-1] ))

In [4]:
def filter_dict_deep_by_keys(adict, list_keys):
    """
    Goal: Filter some Keys of dict even his keys is a sub.key
    BR: Filtrar seomente alguns atributos do JSON, mesmo que seja um Objeto dentro de outro
    Baseado em: https://thispointer.com/python-filter-a-dictionary-by-conditions-on-keys-or-values/
    @list_keys vem dea funcao : 'convert_long_comment_in_string'
    """
    newDict = dict()
    for (key, value) in adict.items():
        if key in list_keys:
            newDict[key] = value
    return newDict            

In [5]:
def request_get(url, lib_parser = 'html.parser'):
    """
    lib_paeser : 'html.parser' // 
    return BeautifulSoupPage
    """
    response = requests.get(url)
    if(response.status_code != 200):
        print(url, 'Erro Code')
        quit()
    return BeautifulSoup(response.text, lib_parser)

In [6]:
def to_df_from_list_of_objs_sorted(list_of_obj):
    """
        Converte de:
    [{'id': 5, 'nome': 'rafael'},  {'id': 6, 'nome': 'marta'}]
        Para DataFrame
    OBS: Esse processo é mais complxo pois tem que ser feito
    """
    variables = list_of_obj[0].keys()

    def aux_obj_to_list_sorted(obj, features):
        alist = []
        for feat in features:
            alist.append(obj[feat])
        return alist

    return pd.DataFrame(
                list( map( lambda x: aux_obj_to_list_sorted(x, variables), list_of_obj)),
                columns = variables)

# Gett All Governadores de um Estado

````
<tbody>
   <tr>
   
      <td class="candidato">
         <a class="clearfix" href="https://especiais.gazetadopovo.com.br/eleicoes/2018/candidatos/ba/governador/rui-costa/">
            <figure class="thumb">
               <img class="lazy lazy-hidden" src="..." data-lazy-type="image" data-lazy-src="..." alt="Candidato Rui Costa 13">
               <noscript><img src="https://cdn-eleicoes.gazetadopovo.com.br/fotos/bahia/governador/rui-costa-min.jpg" alt="Candidato Rui Costa 13"></noscript>
            </figure>
            <div class="nome-partido">
               <h2 class="nome">
                  Rui Costa <small class="label label-success">Eleito</small>	            						
               </h2>
               <span class="partido">PT</span>
            </div>
         </a>
      </td>
      
      <td class="votos">
         <span class="porcentagem">
         75,50%	            				
         </span>
         <span class="qtd-votos">
         5.096.062 votos
         </span>
      </td>
   </tr>
````

In [7]:
base_url = 'https://especiais.gazetadopovo.com.br/eleicoes/2018/resultados/candidatos-mais-votados-para-governador-da-ba/'

In [8]:
soup_gazeta_all_gov = request_get(base_url)

In [10]:
list_tr = soup_gazeta_all_gov.find('tbody').find_all('tr')
gov_list = []

for el in list_tr:
    city_dict = {}
    tds = el.find_all('td')
    city_dict['url_gov'] = tds[0].find('a')['href']
    city_dict['name'] = tds[0].find('h2').get_text().strip()
    city_dict['partido'] = tds[0].find('span', {'class': 'partido'}).get_text().strip()
    city_dict['percentage'] = tds[1].find('span', {'class': 'porcentagem'}).get_text().strip().replace(',','.')
    city_dict['votes'] = tds[1].find('span', {'class': 'qtd-votos'}).get_text().split(' ')[0].strip().replace('.','')
    gov_list.append(city_dict)

In [13]:
gov_list

[{'url_gov': 'https://especiais.gazetadopovo.com.br/eleicoes/2018/candidatos/ba/governador/rui-costa/',
  'name': 'Rui Costa Eleito',
  'partido': 'PT',
  'percentage': '75.50%',
  'votes': '5096062'},
 {'url_gov': 'https://especiais.gazetadopovo.com.br/eleicoes/2018/candidatos/ba/governador/ze-ronaldo/',
  'name': 'Zé Ronaldo',
  'partido': 'DEM',
  'percentage': '22.26%',
  'votes': '1502266'},
 {'url_gov': 'https://especiais.gazetadopovo.com.br/eleicoes/2018/candidatos/ba/governador/marcos-mendes/',
  'name': 'Marcos Mendes',
  'partido': 'PSOL',
  'percentage': '0.66%',
  'votes': '44702'},
 {'url_gov': 'https://especiais.gazetadopovo.com.br/eleicoes/2018/candidatos/ba/governador/joao-henrique-durval-carneiro/',
  'name': 'João Henrique Durval Carneiro',
  'partido': 'PRTB',
  'percentage': '0.58%',
  'votes': '38960'},
 {'url_gov': 'https://especiais.gazetadopovo.com.br/eleicoes/2018/candidatos/ba/governador/joao-santana/',
  'name': 'João Santana',
  'partido': 'MDB',
  'percenta

In [12]:
to_df_from_list_of_objs_sorted(gov_list)

,url_gov,name,partido,percentage,votes
0,https://especiais.gazetadopovo.com.br/eleicoes...,Rui Costa Eleito,PT,75.50%,5096062
1,https://especiais.gazetadopovo.com.br/eleicoes...,Zé Ronaldo,DEM,22.26%,1502266
2,https://especiais.gazetadopovo.com.br/eleicoes...,Marcos Mendes,PSOL,0.66%,44702
3,https://especiais.gazetadopovo.com.br/eleicoes...,João Henrique Durval Carneiro,PRTB,0.58%,38960
4,https://especiais.gazetadopovo.com.br/eleicoes...,João Santana,MDB,0.49%,33266
5,https://especiais.gazetadopovo.com.br/eleicoes...,Célia Sacramento,REDE,0.46%,31198
6,https://especiais.gazetadopovo.com.br/eleicoes...,Orlando Andrade,PCO,0.05%,3100


In [20]:
gov_list_2 = []

for gov in gov_list:
    gov_dict = {}
    print(gov['name'], gov['partido'], '\n\n')
    
    soup_person_gov = request_get(gov['url_gov'])
    section_html = soup_person_gov.find('section', {"class": "tabela-candidato"})

    # Election Situation
    gov_dict['description'] = section_html.find('p').get_text()
    gov_dict['winn'] = section_html.find('span').get_text()
    gov_dict['link_cities'] = section_html.find_all('tr')[-1].td.a['href']
    
    # Get Short Description
    #  <p class="h5 gravata">
    gov_dict['short_description'] = section_html_bio.find(
        'p', {'class': 'h5 gravata'}).get_text().strip().replace('\n','').replace('\t',' ')
    
    # Get Ultra Bio
    try:
        section_html_bio = soup_person_gov.find('div', {"class": "content-sidebar bg-cinza-claro"})
        gov_dict['ultra_description'] = section_html_bio.find(
            'div', {'class': 'bio'}).find('div', {'class': 'text'}).find('p').get_text().strip()
    except:
        print('erro ao buscar "ultra_description"')
        gov_dict['ultra_description'] = ''

    # Bio Info
    section_table = section_html_bio.find('table')
    section_bio_tr = section_table.find_all('tr')
    for el in section_bio_tr:
        gov_dict[el.th.get_text().replace(':','')] = el.td.get_text() 
    
    gov_list_2.append({**gov, **gov_dict})

Rui Costa Eleito PT 


Zé Ronaldo DEM 


Marcos Mendes PSOL 


João Henrique Durval Carneiro PRTB 


João Santana MDB 


Célia Sacramento REDE 


Orlando Andrade PCO 


erro ao buscar "ultra_description"


[{'url_gov': 'https://especiais.gazetadopovo.com.br/eleicoes/2018/candidatos/ba/governador/rui-costa/',
  'name': 'Rui Costa Eleito',
  'partido': 'PT',
  'percentage': '75.50%',
  'votes': '5096062',
  'description': 'Candidato do PT, Rui Costa obteve 5.096.062 votos totalizados (75,50% dos votos válidos) e foi eleito Governador na Bahia nas Eleições 2018.',
  'winn': 'Eleito',
  'link_cities': 'https://especiais.gazetadopovo.com.br/eleicoes/2018/resultados/municipios-bahia/governador-candidato-rui-costa/',
  'short_description': 'Casado(a),\n\t\t\t\t\t\tAgente Postal,                         nascido\t\t\t\t\t\tem 02/05/1985\t\t\t\t\t\tem Feira de Santana-BA,\n\t\t\t\t\t\tcandidato\t\t\t\t\t\ta Governador\t\t\t\t\t\tna Bahia\t\t\t\t\t\tpelo PCO - Partido da Causa Operária',
  'ultra_description': 'Rui Costa dos Santos é o atual governador da Bahia.  Foi um dos fundadores do Partido dos Trabalhadores (PT) na Bahia. Começou sua carreira política no início dos anos 2000, quando foi eleit

In [22]:
df_end = to_df_from_list_of_objs_sorted(gov_list_2)
df_end

,url_gov,name,partido,percentage,votes,description,winn,link_cities,short_description,ultra_description,Nome na urna,Sexo,Idade,Data de nascimento,Ocupação,Grau de instrução,Estado Civil,Cor/Raça,Município de nascimento
0,https://especiais.gazetadopovo.com.br/eleicoes...,Rui Costa Eleito,PT,75.50%,5096062,"Candidato do PT, Rui Costa obteve 5.096.062 vo...",Eleito,https://especiais.gazetadopovo.com.br/eleicoes...,"Casado(a),\n\t\t\t\t\t\tAgente Postal, ...",Rui Costa dos Santos é o atual governador da B...,Rui Costa,Masculino,59 anos,18/01/1963,Economista,Superior completo,Casado(a),Parda,Salvador-BA
1,https://especiais.gazetadopovo.com.br/eleicoes...,Zé Ronaldo,DEM,22.26%,1502266,Candidato ao cargo de Governador na Bahia pelo...,Não foi eleito,https://especiais.gazetadopovo.com.br/eleicoes...,"Casado(a),\n\t\t\t\t\t\tEconomista, ...",José Ronaldo de Carvalho é ex-prefeito de Feir...,Zé Ronaldo,Masculino,70 anos,18/07/1951,Administrador,Superior completo,Casado(a),Branca,Paripiranga-BA
2,https://especiais.gazetadopovo.com.br/eleicoes...,Marcos Mendes,PSOL,0.66%,44702,Candidato ao cargo de Governador na Bahia pelo...,Não foi eleito,https://especiais.gazetadopovo.com.br/eleicoes...,"Casado(a),\n\t\t\t\t\t\tAdministrador, ...",Marcos Antonio Guimarães Mendes é geólogo e po...,Marcos Mendes,Masculino,57 anos,28/02/1965,Outros,Superior completo,Casado(a),Parda,Salvador-BA
3,https://especiais.gazetadopovo.com.br/eleicoes...,João Henrique Durval Carneiro,PRTB,0.58%,38960,Candidato ao cargo de Governador na Bahia pelo...,Não foi eleito,https://especiais.gazetadopovo.com.br/eleicoes...,"Casado(a),\n\t\t\t\t\t\t ...",João Henrique de Barradas Carneiro é candidato...,João Henrique Durval Carneiro,Masculino,62 anos,19/06/1959,Economista,Superior completo,Casado(a),Parda,Feira de Santana-BA
4,https://especiais.gazetadopovo.com.br/eleicoes...,João Santana,MDB,0.49%,33266,Candidato ao cargo de Governador na Bahia pelo...,Não foi eleito,https://especiais.gazetadopovo.com.br/eleicoes...,"Casado(a),\n\t\t\t\t\t\tEconomista, ...",João Reis Santana Filho é engenheiro eletricis...,João Santana,Masculino,78 anos,01/02/1944,Aposentado (Exceto Servidor Público),Ensino Médio completo,Casado(a),Parda,Nazaré-BA
5,https://especiais.gazetadopovo.com.br/eleicoes...,Célia Sacramento,REDE,0.46%,31198,Candidata ao cargo de Governadora na Bahia pel...,Não foi eleita,https://especiais.gazetadopovo.com.br/eleicoes...,"Casado(a),\n\t\t\t\t\t\tAposentado (Exceto Ser...",Célia Sacramento é professora e política. Ela ...,Célia Sacramento,Feminino,54 anos,28/07/1967,Professor de Ensino Superior,Superior completo,Divorciado(a),Preta,São Paulo-SP
6,https://especiais.gazetadopovo.com.br/eleicoes...,Orlando Andrade,PCO,0.05%,3100,Candidato ao cargo de Governador na Bahia pelo...,Não foi eleito,https://especiais.gazetadopovo.com.br/eleicoes...,"Divorciado(a),\n\t\t\t\t\t\tProfessor de Ensin...",,Orlando Andrade,Masculino,36 anos,02/05/1985,Agente Postal,Ensino Médio completo,Casado(a),Preta,Feira de Santana-BA


Inserir coluna Manual que permite ligar com TSE

In [26]:
df_end.insert(loc=1, column="id_tse", value=[
    50000607711, 50000608319, 50000600867, 50000622082, 50000608056, 50000623171, 50000618934]
             )

ValueError: cannot insert id_tse, already exists

In [29]:
df_end

,url_gov,id_tse,name,partido,percentage,votes,description,winn,link_cities,short_description,ultra_description,Nome na urna,Sexo,Idade,Data de nascimento,Ocupação,Grau de instrução,Estado Civil,Cor/Raça,Município de nascimento
0,https://especiais.gazetadopovo.com.br/eleicoes...,50000607711,Rui Costa Eleito,PT,75.50%,5096062,"Candidato do PT, Rui Costa obteve 5.096.062 vo...",Eleito,https://especiais.gazetadopovo.com.br/eleicoes...,"Casado(a),\n\t\t\t\t\t\tAgente Postal, ...",Rui Costa dos Santos é o atual governador da B...,Rui Costa,Masculino,59 anos,18/01/1963,Economista,Superior completo,Casado(a),Parda,Salvador-BA
1,https://especiais.gazetadopovo.com.br/eleicoes...,50000608319,Zé Ronaldo,DEM,22.26%,1502266,Candidato ao cargo de Governador na Bahia pelo...,Não foi eleito,https://especiais.gazetadopovo.com.br/eleicoes...,"Casado(a),\n\t\t\t\t\t\tEconomista, ...",José Ronaldo de Carvalho é ex-prefeito de Feir...,Zé Ronaldo,Masculino,70 anos,18/07/1951,Administrador,Superior completo,Casado(a),Branca,Paripiranga-BA
2,https://especiais.gazetadopovo.com.br/eleicoes...,50000600867,Marcos Mendes,PSOL,0.66%,44702,Candidato ao cargo de Governador na Bahia pelo...,Não foi eleito,https://especiais.gazetadopovo.com.br/eleicoes...,"Casado(a),\n\t\t\t\t\t\tAdministrador, ...",Marcos Antonio Guimarães Mendes é geólogo e po...,Marcos Mendes,Masculino,57 anos,28/02/1965,Outros,Superior completo,Casado(a),Parda,Salvador-BA
3,https://especiais.gazetadopovo.com.br/eleicoes...,50000622082,João Henrique Durval Carneiro,PRTB,0.58%,38960,Candidato ao cargo de Governador na Bahia pelo...,Não foi eleito,https://especiais.gazetadopovo.com.br/eleicoes...,"Casado(a),\n\t\t\t\t\t\t ...",João Henrique de Barradas Carneiro é candidato...,João Henrique Durval Carneiro,Masculino,62 anos,19/06/1959,Economista,Superior completo,Casado(a),Parda,Feira de Santana-BA
4,https://especiais.gazetadopovo.com.br/eleicoes...,50000608056,João Santana,MDB,0.49%,33266,Candidato ao cargo de Governador na Bahia pelo...,Não foi eleito,https://especiais.gazetadopovo.com.br/eleicoes...,"Casado(a),\n\t\t\t\t\t\tEconomista, ...",João Reis Santana Filho é engenheiro eletricis...,João Santana,Masculino,78 anos,01/02/1944,Aposentado (Exceto Servidor Público),Ensino Médio completo,Casado(a),Parda,Nazaré-BA
5,https://especiais.gazetadopovo.com.br/eleicoes...,50000623171,Célia Sacramento,REDE,0.46%,31198,Candidata ao cargo de Governadora na Bahia pel...,Não foi eleita,https://especiais.gazetadopovo.com.br/eleicoes...,"Casado(a),\n\t\t\t\t\t\tAposentado (Exceto Ser...",Célia Sacramento é professora e política. Ela ...,Célia Sacramento,Feminino,54 anos,28/07/1967,Professor de Ensino Superior,Superior completo,Divorciado(a),Preta,São Paulo-SP
6,https://especiais.gazetadopovo.com.br/eleicoes...,50000618934,Orlando Andrade,PCO,0.05%,3100,Candidato ao cargo de Governador na Bahia pelo...,Não foi eleito,https://especiais.gazetadopovo.com.br/eleicoes...,"Divorciado(a),\n\t\t\t\t\t\tProfessor de Ensin...",,Orlando Andrade,Masculino,36 anos,02/05/1985,Agente Postal,Ensino Médio completo,Casado(a),Preta,Feira de Santana-BA


In [30]:
df_end.to_csv('all_governors_from_gazeta.csv', index=False, sep=';', encoding='iso-8859-1')

# Votos Por cidade

````
<tr>
  <td class="candidato">
    <a class="clearfix" href="https://especiais.gazetadopovo.com.br/eleicoes/2018/resultados/municipios-bahia/candeias-ba/deputado-estadual/">
      <div class="nome-partido">
        <h2 class="nome">
          Candeias        						
        </h2>
      </div>
    </a>
  </td>
  <td class="votos">
    <span class="porcentagem">
    15,31%        				</span>
    <span class="qtd-votos">6.556 votos</span>
  </td>
</tr>
````

In [39]:
govs_list_cities = []

for gov in gov_list_2:
    print(gov['name'], gov['partido'], '\n\n')
    
    soup_cities_by_gov = request_get(gov['link_cities'])
    list_tr = soup_cities_by_gov.find('tbody').find_all('tr')
    
    for el in list_tr:
        tds = el.find_all('td')
        gov_city_dict = {}
#         print(tds)
#         break
        # constant by governor
        gov_city_dict['url_source_gov'] =  gov['url_gov']
        gov_city_dict['governor'] =  gov['name']
        # each city
        gov_city_dict['url_the_city'] = tds[0].find('a')['href']
        gov_city_dict['name'] = tds[0].find('h2').get_text().strip()
        gov_city_dict['percentage'] = tds[1].find(
            'span', {'class': 'porcentagem'}).get_text().strip()
        gov_city_dict['votes'] = tds[1].find(
            'span', {'class': 'qtd-votos'}).get_text().split(' ')[0].strip().replace('.','')
        govs_list_cities.append(gov_city_dict)
#     break
        

Rui Costa Eleito PT 


Zé Ronaldo DEM 


Marcos Mendes PSOL 


João Henrique Durval Carneiro PRTB 


João Santana MDB 


Célia Sacramento REDE 


Orlando Andrade PCO 




In [40]:
df_govs_cities = to_df_from_list_of_objs_sorted(govs_list_cities)
df_govs_cities

,url_source_gov,governor,url_the_city,name,percentage,votes
0,https://especiais.gazetadopovo.com.br/eleicoes...,Rui Costa Eleito,https://especiais.gazetadopovo.com.br/eleicoes...,Caetanos,"95,66%",6.089
1,https://especiais.gazetadopovo.com.br/eleicoes...,Rui Costa Eleito,https://especiais.gazetadopovo.com.br/eleicoes...,Souto Soares,"95,17%",8.772
2,https://especiais.gazetadopovo.com.br/eleicoes...,Rui Costa Eleito,https://especiais.gazetadopovo.com.br/eleicoes...,Iraquara,"95,02%",11.077
3,https://especiais.gazetadopovo.com.br/eleicoes...,Rui Costa Eleito,https://especiais.gazetadopovo.com.br/eleicoes...,Dom Basílio,"94,69%",6.228
4,https://especiais.gazetadopovo.com.br/eleicoes...,Rui Costa Eleito,https://especiais.gazetadopovo.com.br/eleicoes...,Jucuruçu,"94,60%",4.710
...,...,...,...,...,...,...
2914,https://especiais.gazetadopovo.com.br/eleicoes...,Orlando Andrade,https://especiais.gazetadopovo.com.br/eleicoes...,Várzea Nova,"0,00%",0
2915,https://especiais.gazetadopovo.com.br/eleicoes...,Orlando Andrade,https://especiais.gazetadopovo.com.br/eleicoes...,Vereda,"0,00%",0
2916,https://especiais.gazetadopovo.com.br/eleicoes...,Orlando Andrade,https://especiais.gazetadopovo.com.br/eleicoes...,Wagner,"0,00%",0
2917,https://especiais.gazetadopovo.com.br/eleicoes...,Orlando Andrade,https://especiais.gazetadopovo.com.br/eleicoes...,Wanderley,"0,00%",0


In [41]:
df_govs_cities.to_csv('all_governors_from_gazeta_by_cities.csv', index=False, sep=';', encoding='iso-8859-1')

## Municipios da Bahia

In [44]:
url_cities_if_state = 'https://especiais.gazetadopovo.com.br/eleicoes/2018/resultados/municipios-bahia/'

In [45]:
soup_all_state_cities = request_get(url_cities_if_state)

In [46]:
# demora 10 min sao 500 request::get
list_cities = []

opt_section = soup_all_state_cities.find_all('optgroup')[1]
list_opts = opt_section.find_all('option')[1:]

## all cities
for el in list_opts:
    city_dict = {}
    city_dict['name'] = el.get_text().strip()
    city_dict['url_city'] = el['value']
    
    city_page = request_get(city_dict['url_city'])
    all_dd = city_page.find('dl', {'class': 'perfil-eleitores'}).find_all('dd')
    
    city_dict['totalVotes'] = all_dd[0].get_text().strip().replace('.','')
    city_dict['mascVotes'] =  all_dd[1].get_text().strip().split(' ')[0].replace('.','')
    city_dict['femnVotes'] = all_dd[2].get_text().strip().split(' ')[0].replace('.','')

    list_cities.append(city_dict)

In [47]:
to_df_from_list_of_objs_sorted(list_cities)

,name,url_city,totalVotes,mascVotes,femnVotes
0,Salvador,https://especiais.gazetadopovo.com.br/eleicoes...,1827436,1022159,805276
1,Abaíra,https://especiais.gazetadopovo.com.br/eleicoes...,6909,3564,3342
2,Abaré,https://especiais.gazetadopovo.com.br/eleicoes...,12713,6404,6302
3,Acajutiba,https://especiais.gazetadopovo.com.br/eleicoes...,11138,5854,5283
4,Adustina,https://especiais.gazetadopovo.com.br/eleicoes...,11118,5776,5342
...,...,...,...,...,...
412,Vitória da Conquista,https://especiais.gazetadopovo.com.br/eleicoes...,221849,120805,101044
413,Wagner,https://especiais.gazetadopovo.com.br/eleicoes...,7206,3635,3571
414,Wanderley,https://especiais.gazetadopovo.com.br/eleicoes...,10198,4927,5268
415,Wenceslau Guimarães,https://especiais.gazetadopovo.com.br/eleicoes...,15576,7737,7837


In [48]:
to_df_from_list_of_objs_sorted(list_cities).to_csv(
    'all_cities_bahia_gov.csv', index=False, sep=';', encoding='iso-8859-1')